直接取回全部銀行指定匯率資訊

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.findrate.tw/USD/"
response = requests.get(url)
data = response.text

soup = BeautifulSoup(data, 'html.parser')
table = soup.find_all('table')[1]

# 提取訊息
exchange_rates = []
# 跳過標題行
for row in table.find_all('tr')[1:]:
    cols = row.find_all('td')
    # 檢查行是否有預期的列數
    if len(cols) == 7:
        exchange_rates.append([col.text.strip() for col in cols])

# 建立 DataFrame
if exchange_rates:
    df = pd.DataFrame(
        exchange_rates, 
        columns=[
            "銀行名稱", "現鈔買入", "現鈔賣出", "即期買入", 
            "即期賣出", "更新時間", "現鈔手續費"
        ]
    )
    # 儲存到 Excel
    df.to_excel("data/美金匯率.xlsx", index=False)
else:
    print("未找到資料或表格格式不同。")


優化以上代碼

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# 目標網址
url = "https://www.findrate.tw/USD/"

# 添加 User-Agent 避免被阻擋
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}

# 發送請求
response = requests.get(url, headers=headers)

# 確保請求成功
if response.status_code == 200:
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')

    # 找到所有表格，避免索引錯誤
    tables = soup.find_all('table')
    
    if len(tables) > 1:
        # 目標表格
        table = tables[1]

        # 提取訊息
        exchange_rates = []
        # 跳過標題行
        for row in table.find_all('tr')[1:]:
            cols = row.find_all('td')
            # 確保符合預期格式
            if len(cols) == 7:
                exchange_rates.append([col.text.strip() for col in cols])

        # 建立 DataFrame
        if exchange_rates:
            df = pd.DataFrame(
                exchange_rates, 
                columns=[
                    "銀行名稱", "現鈔買入", "現鈔賣出", "即期買入", 
                    "即期賣出", "更新時間", "現鈔手續費"
                ]
            )

            # 生成 Excel 檔案名稱
            today_str = datetime.now().strftime('%Y%m%d')
            excel_filename = f"data/美金匯率_{today_str}.xlsx"

            # 儲存到 Excel
            df.to_excel(excel_filename, index=False)

            print(f"✅ 匯率資料已儲存至 {excel_filename}")
        else:
            print("⚠️ 未找到可用的匯率數據。")
    else:
        print("⚠️ 未找到目標表格，請確認網頁結構是否變更。")
else:
    print(f"❌ 無法取得數據，HTTP 狀態碼: {response.status_code}")


✅ 匯率資料已儲存至 data/美金匯率_20250228.xlsx


加入日期

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def fetch_data():
    """從指定網站抓取美金匯率數據"""
    url = "https://www.findrate.tw/USD/"

    # 添加 `User-Agent` 以模擬瀏覽器，避免請求被阻擋
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    # 發送請求
    response = requests.get(url, headers=headers)

    # 檢查請求是否成功
    if response.status_code != 200:
        print(f"❌ 無法取得數據，HTTP 狀態碼: {response.status_code}")
        return []

    # 解析 HTML 內容
    soup = BeautifulSoup(response.text, 'html.parser')

    # 嘗試提取日期資訊
    date_info = soup.find('span', style="float:right")
    if not date_info:
        print("⚠️ 無法找到更新日期，請確認網頁結構是否變更。")
        return []

    # 解析日期
    date_text = date_info.text.strip().split('時間：')[-1]
    # 移除 `-` 轉為 YYYYMMDD 格式
    date_str = date_text.replace('-', '')

    # 找到所有表格，避免索引錯誤
    tables = soup.find_all('table')
    if len(tables) < 2:
        print("⚠️ 未找到目標表格，請確認網頁結構是否變更。")
        return []

    # 目標表格
    table = tables[1]

    # 提取匯率數據
    exchange_rates = []
    # 跳過標題行
    for row in table.find_all('tr')[1:]:
        cols = row.find_all('td')
        # 確保行數符合預期
        if len(cols) == 7:
            row_data = [col.text.strip() for col in cols]

            # 處理更新時間，移除 HTML 註解並合併日期
            update_time = BeautifulSoup(row_data[5], "html.parser").text
            # 完整時間資訊
            row_data[5] = f"{date_text} {update_time}"
            exchange_rates.append(row_data)
    # 回傳數據與日期字串
    return exchange_rates, date_str

# 抓取數據
exchange_rates, date_str = fetch_data()

if exchange_rates:
    # 建立 DataFrame
    df = pd.DataFrame(
        exchange_rates, 
        columns=[
            "銀行名稱", "現鈔買入", "現鈔賣出", "即期買入",
            "即期賣出", "更新時間", "現鈔手續費"
        ]
    )

    # 生成 Excel 檔案名稱
    excel_filename = f"data/美金匯率_V2_{date_str}.xlsx"

    # 儲存為 Excel 檔案
    df.to_excel(excel_filename, index=False)

    print(f"✅ 匯率資料已儲存至 {excel_filename}")
else:
    print("⚠️ 沒有找到可用的數據。")


✅ 匯率資料已儲存至 data/美金匯率_V2_20250228.xlsx
